**Download data**

In [ ]:
import requests

url = "https://github.com/vharatian/MLIntrodcutionWorkshop/raw/main/all_stocks_5yr.csv"
file_address = "/content/data.csv"

response = requests.get(url)

if response.status_code == 200:
    with open(file_address, 'wb') as file:
        file.write(response.content)
    print(f"File '{file_address}' downloaded successfully.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

**Load data and filter Apple**

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_address)
df = df.loc[df['Name'] == 'AAPL']
df.info()

In [ ]:
df2 = df['close']
df2 = pd.DataFrame(df2)

future_days = 100
df2['Prediction'] = df2['close'].shift(-future_days)

X = np.array(df2.drop(['Prediction'], 1))[:-future_days]
y = np.array(df2['Prediction'])[:-future_days]

**Split the data and create models**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
x_future = df2.drop(['Prediction'], 1)[:-future_days]
x_future = x_future.tail(future_days)
x_future = np.array(x_future)

**Draw prediction function**

In [72]:
def draw_prediction(predictions):
  valid = df2[X.shape[0]:]
  valid['Predictions'] = predictions

  plt.figure(figsize=(16,8))
  plt.title("Model")
  plt.xlabel('Days')
  plt.ylabel('Close Price USD ($)')
  plt.plot(df2['close'])
  plt.plot(valid[['close', 'Predictions']])
  plt.legend(["Original", "Valid", 'Predicted'])
  plt.show()

**Predict using linear regression**

In [ ]:
lr = LinearRegression().fit(x_train, y_train)
lr_prediction = lr.predict(x_future)

draw_prediction(lr_prediction)

**Improve liner model**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_reg = PolynomialFeatures(degree = 3)
X_polynomial = poly_reg.fit_transform(x_train)
lr_pol = LinearRegression()
lr_pol.fit(X_polynomial, y_train)
lr_pol_prediction = lr_pol.predict(poly_reg.fit_transform(x_future))

draw_prediction(lr_pol_prediction)

**Predict using decision tree**

In [ ]:
tree = DecisionTreeRegressor().fit(x_train, y_train)

tree_prediction = tree.predict(x_future)
draw_prediction(tree_prediction)